# report

1.EDA

2.预处理

暂时先做最简单的， NA->-1, Dummy 所有cata且个数<10的， std没做

3.BaseModel

Boosting Trees

RF

GridSearch

Fine Tuned Accuracy

    XGB
        param = {'eta': 0.02, 
                'max_depth': 4, 
                'subsample': 0.9, 
                'colsample_bytree': 0.9, 
                'objective': 'binary:logistic', 
                'eval_metric': 'auc', 
                'seed': 99, 
                'silent': True
               }  
    cv_accuracy = 0.2835
    test = 0.280

    LGBM1
        param = {
            'learning_rate' : 0.02,
            'n_estimators' : 650,
            'max_bin' : 10,
            'subsample' : 0.8,
            'subsample_freq' : 10,
            'colsample_bytree' : 0.8,
            'min_child_samples' : 500,
            'seed' : 99,
            }
    cv_accuracy = 0.2844
    test = 0.281

    LGBM2
        param = {
                'learning_rate' : 0.02,
                'n_estimators' : 1090,
                'subsample' : 0.7,
                'subsample_freq' : 2,
                'num_leaves' : 16,
                'seed' : 99,
                }
    cv_accuracy = 0.2833               
    test = 0.281

    RF
        param = {'n_estimators': 150,
                    'criterion': 'gini',
                    'max_features': 'log2',
                    'max_depth': 9,
                    'min_samples_split': 70,
                    'min_samples_leaf': 30,
                    'n_jobs' : -1,
                    'max_leaf_nodes': None,
                    'bootstrap': True,
                    'oob_score': True,
                    'random_state': 99,
                    'verbose': 0,
                    'warm_start': True,
                    'class_weight': {0: 0.0364, 1: 0.9635},
                    }
    cv_accuracy = 0.2589                 

4.Stacking

    stacking
        all_50%      : 0.272
        all_weighted : 0.273
        3_weighted   : 0.281
        3_50%        : 0.282

5.预处理测略

    NA的列都删了？
        xgboost : 0.2529
        lgbm1   : 0.2502
        lgbm2   : 0.2510

std

NA

类别没有被Dummy的进行分析

In [4]:
import pandas as pd
import numpy as np
from modeling import dataset
train = pd.read_csv("./original/train.csv")
test = pd.read_csv("./original/test.csv")
# drop unrelated items
train = dataset(train.drop("id", 1).drop("target", 1),
                train.target.to_frame('target'))
test = dataset(test.drop("id", 1), test.id.to_frame('id'))
# -1 are actually NaN
train.X = train.X.replace(-1, np.nan)
test.X = test .X.replace(-1, np.nan)

In [8]:
for col in train.X.columns:
    if train.X[col].hasnans:
        print(col)

ps_ind_02_cat
ps_ind_04_cat
ps_ind_05_cat
ps_reg_03
ps_car_01_cat
ps_car_02_cat
ps_car_03_cat
ps_car_05_cat
ps_car_07_cat
ps_car_09_cat
ps_car_11
ps_car_12
ps_car_14
